In [1]:
import numpy as np
import pandas as pd

#df_train = pd.read_csv("../Kaggle/House-Prices/train.csv")
df_train = pd.read_csv("house-prices.csv")

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

df_train.head()

In [2]:
lista = (df_train.dtypes == 'object')
object_cols = list(lista[lista].index)

print("Variáveis Categóricas:")
print(object_cols)

Variáveis Categóricas:
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [3]:
lista = (df_train.dtypes == 'float')
object_cols = list(lista[lista].index)

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds) #Quanto Menor, Melhor!!!

from sklearn.model_selection import train_test_split
y = df_train.SalePrice
X = df_train.drop(['SalePrice'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Drop Columns

In [5]:
from sklearn.preprocessing import LabelEncoder

# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

print(cols_with_missing);print()

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

# Make copy to avoid changing original data 
label_X_train = reduced_X_train.copy()
label_X_valid = reduced_X_valid.copy()

lista = (reduced_X_train.dtypes == 'object')
object_cols = list(lista[lista].index)

label_encoder = LabelEncoder()
for col in object_cols:
    _encoder = label_encoder.fit(pd.concat([reduced_X_valid[col], reduced_X_train[col]], axis=0))
    label_X_valid[col] = _encoder.transform(reduced_X_valid[col])
    label_X_train[col] = _encoder.transform(reduced_X_train[col])

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

MAE from Approach 1 (Drop columns with missing values):
17181.396130136985


# Imputer: Mean, Tracking Column

In [6]:
def fill_all_columns(df, verbose=False):
    from sklearn.preprocessing import LabelEncoder
    
    for column in df.columns:
        if df[column].isnull().values.any():
            '''if df[column].dtype == 'float64':
                df.fillna(value={column: df[column].mean()}, inplace=True)
            el'''
            if df[column].dtype == 'int64' or df[column].dtype == 'object':
                df[column].fillna(method='bfill', inplace=True)
                df[column].fillna(method='ffill', inplace=True)
                
        if df[column].dtype == 'object':
            encoder_ = LabelEncoder().fit(df[column])
            df[column] = encoder_.transform(df[column])
                
    if verbose:
        df.info()

In [7]:
#from sklearn.preprocessing.Imputer import SimpleImputer #Versões mais novas
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

fill_all_columns(X_train_plus)
fill_all_columns(X_valid_plus)

# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
#my_imputer = SimpleImputer() #Versões mais novas
my_imputer = Imputer()
_imputer = my_imputer.fit(pd.concat([X_train_plus, X_valid_plus], axis = 0))
imputed_X_train_plus = pd.DataFrame(_imputer.transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
17704.707294520547
